# Машинное обучение, ФКН ВШЭ

## Практическое задание 8. Метод опорных векторов и аппроксимация ядер

### Общая информация
Дата выдачи: 15.03.2020

Мягкий дедлайн: 02:59MSK 30.03.2020

Жесткий дедлайн: 23:59MSK 03.04.2020

### Оценивание и штрафы
Каждая из задач имеет определенную «стоимость» (указана в скобках около задачи). Максимальная оценка за работу (без учёта бонусов) — 10 баллов.

Сдавать задание после указанного жёсткого срока сдачи нельзя. При выставлении неполного балла за задание в связи с наличием ошибок на усмотрение проверяющего предусмотрена возможность исправить работу на указанных в ответном письме условиях.

Задание выполняется самостоятельно. «Похожие» решения считаются плагиатом и все задействованные студенты (в том числе те, у кого списали) не могут получить за него больше 0 баллов (подробнее о плагиате см. на странице курса). Если вы нашли решение какого-то из заданий (или его часть) в открытом источнике, необходимо указать ссылку на этот источник в отдельном блоке в конце вашей работы (скорее всего вы будете не единственным, кто это нашел, поэтому чтобы исключить подозрение в плагиате, необходима ссылка на источник).

Неэффективная реализация кода может негативно отразиться на оценке.

### Формат сдачи
Задания сдаются через систему anytask. Посылка должна содержать:
* Ноутбук homework-practice-08-random-features-Username.ipynb

Username — ваша фамилия и имя на латинице именно в таком порядке

### О задании

На занятиях мы подробно обсуждали метод опорных векторов (SVM). В базовой версии в нём нет чего-то особенного — мы всего лишь используем специальную функцию потерь, которая не требует устремлять отступы к бесконечности; ей достаточно, чтобы отступы были не меньше +1. Затем мы узнали, что SVM можно переписать в двойственном виде, который, позволяет заменить скалярные произведения объектов на ядра. Это будет соответствовать построению модели в новом пространстве более высокой размерности, координаты которого представляют собой нелинейные модификации исходных признаков.

Ядровой SVM, к сожалению, довольно затратен по памяти (нужно хранить матрицу Грама размера $d \times d$) и по времени (нужно решать задачу условной оптимизации с квадратичной функцией, а это не очень быстро). Мы обсуждали, что есть способы посчитать новые признаки $\tilde \varphi(x)$ на основе исходных так, что скалярные произведения этих новых $\langle \tilde \varphi(x), \tilde \varphi(z) \rangle$ приближают ядро $K(x, z)$.

Мы будем исследовать аппроксимации методом Random Kitchen Sinks для гауссовых ядер. Будем использовать формулы, которые немного отличаются от того, что было на лекциях (мы добавим сдвиги внутрь тригонометрических функций и будем использовать только косинусы, потому что с нужным сдвигом косинус превратится в синус):
$$\tilde \varphi(x) = (
\cos (w_1^T x + b_1),
\dots,
\cos (w_n^T x + b_n)
),$$
где $w_j \sim \mathcal{N}(0, 1/\sigma^2)$, $b_j \sim U[-\pi, \pi]$.

На новых признаках $\tilde \varphi(x)$ мы будем строить любую линейную модель.

Можно считать, что это некоторая новая парадигма построения сложных моделей. Можно направленно искать сложные нелинейные закономерности в данных с помощью градиентного бустинга или нейронных сетей, а можно просто нагенерировать большое количество случайных нелинейных признаков и надеяться, что быстрая и простая модель (то есть линейная) сможет показать на них хорошее качество. В этом задании мы изучим, насколько работоспособна такая идея.

### Алгоритм

Вам потребуется реализовать следующий алгоритм:
1. Понизить размерность выборки до new_dim с помощью метода главных компонент.
2. Для полученной выборки оценить гиперпараметр $\sigma^2$ с помощью эвристики (рекомендуем считать медиану не по всем парам объектов, а по случайному подмножеству из где-то миллиона пар объектов): $$\sigma^2 = \text{median}_{i, j = 1, \dots, \ell, i \neq j} \left\{\sum_{k = 1}^{d} (x_{ik} - x_{jk})^2 \right\}$$
3. Сгенерировать n_features наборов весов $w_j$ и сдвигов $b_j$.
4. Сформировать n_features новых признаков по формулам, приведённым выше.
5. Обучить линейную модель (логистическую регрессию или SVM) на новых признаках.
6. Повторить преобразования (PCA, формирование новых признаков) к тестовой выборке и применить модель.

Тестировать алгоритм мы будем на данных Fashion MNIST. Ниже код для их загрузки и подготовки.

In [2]:
import keras
from keras.datasets import fashion_mnist
(x_train_pics, y_train), (x_test_pics, y_test) = fashion_mnist.load_data()

Using TensorFlow backend.


In [3]:
x_train = x_train_pics.reshape(x_train_pics.shape[0],-1).astype('float32')
x_test = x_test_pics.reshape(x_test_pics.shape[0],-1).astype('float32')

In [4]:
from sklearn.decomposition import PCA
import numpy as np
from numpy.random import randint
from scipy.stats import norm, expon, beta, lognorm, uniform
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn import svm
from lightgbm import LGBMClassifier

__Задание 1. (5 баллов)__

Реализуйте алгоритм, описанный выше. Желательно в виде класса с методами fit() и predict().

Что должно быть в вашей реализации:
1. Возможность задавать значения гиперпараметров new_dim (по умолчанию 50) и n_features (по умолчанию 1000).
2. Возможность включать или выключать предварительное понижение размерности с помощью метода главных компонент.
3. Возможность выбирать тип линейной модели (логистическая регрессия или SVM с линейным ядром).

Протестируйте на данных Fashion MNIST, сформированных кодом выше. Если на тесте у вас получилась доля верных ответов не ниже 0.84 с гиперпараметрами по умолчанию, то вы всё сделали правильно.

In [5]:
class PCA_RKS:
    def __init__(self, method, new_dim=50, n_features=1000, isPCA=True):
        self.new_dim = new_dim
        self.n_features = n_features
        self.isPCA = isPCA
        self.method = method
        self.W = None
        self.b = None
        self.dec = None
    
    def PCA(self, X):
        dec = PCA(n_components=self.new_dim)
        dec.fit(X)
        self.dec = dec
        return dec.transform(X)
    
    def transform(self, X):
        if self.dec is not None:
            X = self.dec.transform(X)
        X = np.sqrt(2/self.n_features) * np.cos(self.W.T.dot(X.T).T + self.b)
        return X
        
    def fit(self, X, y):
        if self.isPCA:
            X = self.PCA(X)
        
        C = 10 ** 6
        l = X.shape[0]

        pairs = randint(0, l, (2,C))
        
        sums = []
        for ind in range(C):
            i, j = pairs[0][ind], pairs[1][ind]
            if i != j:
                sums.append(np.sum(np.square(X[i] - X[j])))
                
        sigma2 = np.median(sums)
        self.b = np.random.uniform(low=-np.pi, high=np.pi, size=self.n_features)
        self.W = np.random.normal(loc=0, scale=(1/sigma2)**0.5, size=(X.shape[1], self.n_features))
        X = np.sqrt(2/self.n_features) * np.cos(self.W.T.dot(X.T).T + self.b)
        
        return self.method.fit(X, y)
        
    def predict(self, X):
        return self.method.predict(X)
    
    def score(self, X, y):
        return self.method.score(X, y)


In [14]:
method = svm.LinearSVC()
testing = PCA_RKS(method)

testing.fit(x_train, y_train)

LinearSVC(C=1.0, class_weight=None, dual=True, fit_intercept=True,
          intercept_scaling=1, loss='squared_hinge', max_iter=1000,
          multi_class='ovr', penalty='l2', random_state=None, tol=0.0001,
          verbose=0)

In [15]:
test = testing.transform(x_test)

In [16]:
y_pred = testing.predict(test)
y_pred

array([9, 2, 1, ..., 8, 1, 5], dtype=uint8)

In [17]:
testing.score(test, y_test)

0.8605

Как видим, качество > 0.84. Значит

In [18]:
method2 = LogisticRegression()
testing2 = PCA_RKS(method2)

testing2.fit(x_train, y_train)

/home/dilara/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/home/dilara/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:469: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)


LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='warn', n_jobs=None, penalty='l2',
                   random_state=None, solver='warn', tol=0.0001, verbose=0,
                   warm_start=False)

In [19]:
test2 = testing2.transform(x_test)
y_pred = testing2.predict(test2)
y_pred

array([9, 2, 1, ..., 8, 1, 5], dtype=uint8)

In [20]:
testing2.score(test2, y_test)

0.8319

__Задание 2. (3 балла)__

Сравните подход со случайными признаками с обучением SVM на исходных признаках. Попробуйте вариант с обычным (линейным) SVM и с ядровым SVM. Ядровой SVM может очень долго обучаться, поэтому можно делать любые разумные вещи для ускорения: брать подмножество объектов из обучающей выборки, например.

Сравните подход со случайными признаками с вариантом, в котором вы понижаете размерность с помощью PCA и обучаете градиентный бустинг (используйте нормальную имплементацию, а не из sklearn, и подберите число деревьев и длину шага).

Сделайте выводы — насколько идея со случайными признаками работает? Сравните как с точки зрения качества, так и с точки зрения скорости обучения и применения.

1. Линейный SVM на исходных признаках

In [14]:
svm1 = svm.LinearSVC()
svm1.fit(x_train, y_train)

/home/dilara/anaconda3/lib/python3.7/site-packages/sklearn/svm/base.py:929: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)


LinearSVC(C=1.0, class_weight=None, dual=True, fit_intercept=True,
          intercept_scaling=1, loss='squared_hinge', max_iter=1000,
          multi_class='ovr', penalty='l2', random_state=None, tol=0.0001,
          verbose=0)

In [15]:
svm1.score(x_test, y_test)

0.7589

2. Ядровой SVM на исходных признаках

In [6]:
inds = randint(0, x_train.shape[0], 30000)

In [13]:
x_tr = x_train[:25000]
y_tr = y_train[:25000]

In [14]:
%%time
svm2 = svm.SVC()
svm2.fit(x_tr, y_tr)

/home/dilara/anaconda3/lib/python3.7/site-packages/sklearn/svm/base.py:193: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)


CPU times: user 25min 37s, sys: 3.21 s, total: 25min 40s
Wall time: 25min 42s


SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma='auto_deprecated',
    kernel='rbf', max_iter=-1, probability=False, random_state=None,
    shrinking=True, tol=0.001, verbose=False)

In [15]:
svm2.score(x_test, y_test)

0.1001

Предполагалось, что качество ядрового svm на исходных признаках будет несильно отличаться от линейного svm на случайных признаках, потому что линейное ядро случайных признаков аппроксимирует гауссово ядро на исходных. Но в моем случае качество на исходных получилось значительно хуже

3. Ядровой SVM на случайных признаках

In [43]:
%%time

svm3 = PCA_RKS(svm.SVC())
svm3.fit(x_train[inds], y_train[inds])

/home/dilara/anaconda3/lib/python3.7/site-packages/sklearn/svm/base.py:193: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)


CPU times: user 38min 19s, sys: 1.44 s, total: 38min 21s
Wall time: 38min 18s


SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma='auto_deprecated',
    kernel='rbf', max_iter=-1, probability=False, random_state=None,
    shrinking=True, tol=0.001, verbose=False)

In [44]:
svm3.score(svm3.transform(x_test), y_test)

0.5984

Таким образом, линейные методы работают в разы быстрее, а в комбинации со случайными признаками не уступают в качестве нелинейным методам на исходных признаках (которые в свою очередь работают долго).

3. PCA и Градиентный бустинг

In [27]:
dec = PCA(n_components=50)
dec.fit(x_train)
X_train = dec.transform(x_train)

In [28]:
X_test = dec.transform(x_test)

In [29]:
model = LGBMClassifier()
param_dist = {"n_estimators": [200, 500, 700],
              "learning_rate": [0.1, 0.16, 0.05]
             }

In [31]:
%%time
from sklearn.model_selection import GridSearchCV

grid_search = GridSearchCV(model, param_grid=param_dist, cv = 3, n_jobs=-1)
grid_search.fit(X_train, y_train)

CPU times: user 7min 57s, sys: 796 ms, total: 7min 58s
Wall time: 34min 22s


GridSearchCV(cv=3, error_score='raise-deprecating',
             estimator=LGBMClassifier(boosting_type='gbdt', class_weight=None,
                                      colsample_bytree=1.0,
                                      importance_type='split',
                                      learning_rate=0.1, max_depth=-1,
                                      min_child_samples=20,
                                      min_child_weight=0.001,
                                      min_split_gain=0.0, n_estimators=100,
                                      n_jobs=-1, num_leaves=31, objective=None,
                                      random_state=None, reg_alpha=0.0,
                                      reg_lambda=0.0, silent=True,
                                      subsample=1.0, subsample_for_bin=200000,
                                      subsample_freq=0),
             iid='warn', n_jobs=-1,
             param_grid={'learning_rate': [0.1, 0.16, 0.05],
                         

In [34]:
grid_search.best_estimator_.get_params()

{'boosting_type': 'gbdt',
 'class_weight': None,
 'colsample_bytree': 1.0,
 'importance_type': 'split',
 'learning_rate': 0.16,
 'max_depth': -1,
 'min_child_samples': 20,
 'min_child_weight': 0.001,
 'min_split_gain': 0.0,
 'n_estimators': 700,
 'n_jobs': -1,
 'num_leaves': 31,
 'objective': None,
 'random_state': None,
 'reg_alpha': 0.0,
 'reg_lambda': 0.0,
 'silent': True,
 'subsample': 1.0,
 'subsample_for_bin': 200000,
 'subsample_freq': 0}

In [37]:
model1 = LGBMClassifier(learning_rate=0.16, n_estimators=700)
model1.fit(X_train, y_train)

LGBMClassifier(boosting_type='gbdt', class_weight=None, colsample_bytree=1.0,
               importance_type='split', learning_rate=0.16, max_depth=-1,
               min_child_samples=20, min_child_weight=0.001, min_split_gain=0.0,
               n_estimators=700, n_jobs=-1, num_leaves=31, objective=None,
               random_state=None, reg_alpha=0.0, reg_lambda=0.0, silent=True,
               subsample=1.0, subsample_for_bin=200000, subsample_freq=0)

In [38]:
model1.score(X_test, y_test)

0.8808

Как видим, качество такого подхода (pca+бустинг) намного лучше. Более того с точки зрения скорости обучения, данный метод намного быстрее(если исключить часть с подбором параметров) ядрового svm.
Можно сделать вывод, что идея со случайными признаками в качестве входных данных для линейных методов позволяет сократить время обучения на крупных датасетах, при этом не уступая в качестве ядровым нелинейным методам.

__Задание 3. (2 балла)__

Проведите эксперименты:
1. Помогает ли предварительное понижение размерности с помощью PCA? 
2. Как зависит итоговое качество от n_features? Выходит ли оно на плато при росте n_features?
3. Важно ли, какую модель обучать — логистическую регрессию или SVM?

1. При предварительном понижении размерности доля правильных ответов составляла 0.86.
    Посмотрим, что будет, если отключить предварительное понижение размерности:

In [6]:
method = svm.LinearSVC()

In [24]:
test1 = PCA_RKS(method, isPCA=False)
test1.fit(x_train, y_train)

LinearSVC(C=1.0, class_weight=None, dual=True, fit_intercept=True,
          intercept_scaling=1, loss='squared_hinge', max_iter=1000,
          multi_class='ovr', penalty='l2', random_state=None, tol=0.0001,
          verbose=0)

In [25]:
test = test1.transform(x_test)
test1.score(test, y_test)

0.8683

Результат лучше, но несильно.

2. Посмотрим на качество при различных n_features.

In [47]:
ns = [10, 100, 1000, 2000, 5000]
for n in ns:
    test2 = PCA_RKS(method, n_features=n)
    test2.fit(x_train, y_train)
    test = test2.transform(x_test)
    print('n: ', n, 'accuracy: ', test2.score(test, y_test))

n:  10 accuracy:  0.6213
n:  100 accuracy:  0.831
n:  1000 accuracy:  0.8595
n:  2000 accuracy:  0.8603
n:  5000 accuracy:  0.8629


In [ ]:
ns = [6000, 7000]
for n in ns:
    test2 = PCA_RKS(method, n_features=n)
    test2.fit(x_train, y_train)
    test = test2.transform(x_test)
    print('n: ', n, 'accuracy: ', test2.score(test, y_test))

n:  6000 accuracy:  0.8628


Далее, при больших размерах n_features мой кернел умирает, но в целом видно, что при n_features < 1000  растет довольно стремительно, далее на отрезке [1000, 5000] оно все еще растет, но совсем незначительно, а после выходит на плато.

3. svm vs logreg

In [396]:
method = svm.LinearSVC()
testing = PCA_RKS(method)

testing.fit(x_train, y_train)
test = testing.transform(x_test)
testing.score(test, y_test)

0.8589

In [397]:
method = LogisticRegression()
testing2 = PCA_RKS(method)

testing2.fit(x_train, y_train)
test2 = testing2.transform(x_test)
testing2.score(test2, y_test)

/home/dilara/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/home/dilara/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:469: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)


0.8319

Да, svm справляется лучше logreg на случайных признаках. так как регрессия значительно быстрее, думаю она больше подходит для аппроксимации результатов для более сложных методов

### Бонус

Ниже приведены задания на бонусные баллы. За исследования на стандартных датасетах из sklearn или, скажем, на титанике, баллы не будут начисляться. Приветствуются интересные выводы из проведённых экспериментов.

__Задание 4. (Максимум 2 балла)__

Возьмите какой-нибудь достаточно сложный датасет, на котором хорошо работает градиентный бустинг. Сравните бустинг с нашим алгоритмом со случайными признаками с точки зрения качества и скорости. Подберите как следует гиперпараметры алгоритма.

__Задание 5. (Максимум 2 балла)__

Возьмите какой-нибудь датасет с текстами и решите одним из стандартных методов (например, tf-idf + логистическая регрессия или что-то нейросетевое). Сравните по качеству и скорости с нашим алгоритмом.

__Задание 6. (Максимум 2 балла)__

Поэкспериментируйте на нескольких датасетах из прошлых пунктов с фукнциями для вычисления новых случайных признаков. Не обязательно использовать косинус от скалярного произведения — можно брать знак от него, хэш и т.д. Придумайте побольше вариантов для генерации признаков и проверьте, не получается ли с их помощью добиваться более высокого качества.